## Lobes

- Measure fluxes from different regions at multiple frequencies
- The r.m.s. in the input images should be measured in a different notebook and saved to the input table, since for the Planck images we use a different method.
- MeerKAT observations must be corrected for the primary beam BEFORE measuring the flux
- Planck observations must be converted to the correct units BEFORE measuring the flux
- **Delete** old flux tables before repeaing measurement


In [2]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable



fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/'
imDir = rootDir+'inIms/'
regDir = imDir+'regions/diff_regions/'
inTabs = rootDir+'inTabs/'

names = ['ToT','ELobe','WLobe']

ToTDir = rootDir+names[0]+'/'


aut = 'FgCv'

synToTtable = ToTDir+'synageFl'+'ToT'+aut+'.tbl'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir


tableName = inTabs+'inputFluxTableFgCv.tbl'
tableFileNames= ascii.read(tableName,format='csv')

noiseTableName = inTabs+'noiseFluxIntFgCv.tbl'
noiseValues= ascii.read(noiseTableName)


frList = [ 84.0,114.5,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,100000.0,143000.0]
print frList
print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


[84.0, 114.5, 200.5, 320.0, 1029.9794921, 1440.5292968, 1500.0, 5700.0, 5900.0, 6100.0, 6300.0, 6500.0, 6700.0, 6874.0, 100000.0, 143000.0]
	+---------+
	 Inputs loaded
	+---------+


In [4]:
lobe = ['ee','ww']
lobeName = ['E','W']
regions = ['bridge.reg',  'Ehot.reg',  'Ejet.reg',
           'Enorth.reg',  'Esouth.reg', 'Ecentre.reg', 'Whot.reg', 
           'Wjet.reg',  'Wnorth.reg',  'Wsouth.reg', 'Wcentre.reg']
regName = ['bridge',  'Eh',  'Ej',
           'En',  'Es', 'Ec',  'Wh', 
           'Wj',  'Wn',  'Ws', 'Wc']
#regions = [totReg, elReg, wlReg]


flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)


print '\t Executing FileList+Region Combo' 

fr=[]
frSyn =[]
flIntArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
flErrArray= np.zeros([len(regions),len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

    
for i in xrange(0,len(tableFileNames.columns[0])):
    fileName = imDir+tableFileNames.columns[0][i]

    #fileName = fint.cleanHead(fileName)
    print fileName
    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)

    #if not 'CDELT2' in heads:
    #    heads['CDELT2'] = heads['CD2_2']
    #    heads['CDELT1'] = heads['CD1_1']
    #if 'Frequency' in tableFileNames.dtype.names:
    #    freq = float(tableFileNames.columns[1][i])*1e6
    #    fr.append(freq)
    #elif 'FREQ' in heads:
    #    freq = float(heads['FREQ'])
    #    fr.append(freq)
    #elif 'CRVAL3' in heads:
    #    freq = float(heads['CRVAL3'])
    #    fr.append(freq)
    #else:
    freq = frList[i]*1e6
    fr.append(freq)
    noiseLobe = noiseValues.columns[4][i]

    for j in xrange (0,len(regions)):
        region = regDir+regions[j]
        #print region
        outTable = ToTDir+regName[j]+'FlReg.tbl'

        maskedData, background, noise, pixels = fint.maskDatReg(datas,heads,region,cutoff[j])
        fluxint, numPixBeam =fint.measFlux(maskedData,heads,errPerc[i],'none')
        fluxint *= flCorr[i]
        noise = noiseLobe*pixels/numPixBeam
        flIntArray[j,i] = fluxint
        t,flErr = fint.writeTable(heads,fluxint,noise,numPixBeam,freq,errPerc[i],outTable)
        flErrArray[j,i] = flErr

fr1 =np.asarray(fr)
#if k==0:
frSyn = fr1.copy()

for j in xrange (0,len(regions)):
    synLTable = ToTDir+'synageFl'+regName[j]+'Reg.tbl'
    tName = syn.writeSynageTable(synLTable,frSyn,flIntArray[j,:],flErrArray[j,:],regName[j])


print '''\t+---------+\n\t Fluxes measured\n\t Tables Saved\n\t+---------+'''
  

	 Executing FileList+Region Combo
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/CONVL/CONVL_MWA_84MHz_bt32_corr.fits
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/CONVL/CONVL_MWA_105MHz_bt32_corr.fits
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/CONVL/CONVL_MWA_200MHz_bt32_corr.fits
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
0.16111111111111112
/home/maccagni/Projects/MFS/FornaxA/cont